In [10]:
# Import required libraries
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the Spacy language model for POS tagging
nlp = spacy.load("en_core_web_sm")

# Load the AG News dataset from Hugging Face
dataset = load_dataset("ag_news")
train_data = dataset['train']
X_train = train_data['text']

# Apply TF-IDF vectorization to the text data
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
vectorizer.fit(X_train)

# Function to filter out verbs from the text
def filter_verbs(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if token.pos_ != "VERB"])

# Function to recommend hashtags
def recommend_hashtags(text, vectorizer, top_n=5):
    filtered_text = filter_verbs(text)
    tfidf_matrix = vectorizer.transform([filtered_text])
    tfidf_array = tfidf_matrix.toarray().flatten()
    top_indices = np.argsort(tfidf_array)[-top_n:]
    feature_names = vectorizer.get_feature_names_out()
    top_terms = [feature_names[i] for i in top_indices]
    hashtags = [f"#{term}" for term in top_terms]
    return top_terms, hashtags

# Create widgets for input and output
text_input = widgets.Textarea(
    description='Input Text:',
    placeholder='Enter your text here...',
    layout=widgets.Layout(width='100%', height='100px')
)

output = widgets.Output()

# Define a function to handle button click
def on_button_click(b):
    with output:
        clear_output()  # Clear previous output
        input_text = text_input.value
        top_terms, hashtags = recommend_hashtags(input_text, vectorizer)

        print(f"Top Terms: {', '.join(top_terms)}")
        print(f"Hashtag Recommendations: {', '.join(hashtags)}")

# Create a button widget
button = widgets.Button(
    description='Get Recommendations',
    button_style='success'
)

# Link button click to the function
button.on_click(on_button_click)

# Display the widgets
display(text_input, button, output)


Textarea(value='', description='Input Text:', layout=Layout(height='100px', width='100%'), placeholder='Enter …

Button(button_style='success', description='Get Recommendations', style=ButtonStyle())

Output()